<a href="https://colab.research.google.com/github/Yosoy1408/CasasParaTodosBot/blob/main/CasasParaTodosBot_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
!pip install python-telegram-bot==13.7
!pip install pandas numpy matplotlib
!pip install Pillow
!pip install requests
!pip install firebase-admin
!pip install python-dotenv
!pip install schedule
!pip install emoji
!pip install plotly
!pip install seaborn
!pip install opencv-python-headless
!pip install python-dateutil
!pip install pytz

In [33]:
import os
import json
import sqlite3
import logging
import random
import time
import schedule
import emoji
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import pytz
from PIL import Image, ImageDraw, ImageFont
from io import BytesIO
from telegram import Update, InlineKeyboardButton, InlineKeyboardMarkup, ParseMode
from telegram.ext import (
    Updater,
    CommandHandler,
    CallbackQueryHandler,
    CallbackContext,
    MessageHandler,
    Filters,
    ConversationHandler
)
from threading import Thread

In [34]:
# Token del bot
TOKEN = "7643294498:AAEJM1ftebsJmN1Jt2-lXkKA3l7bMcd7n0Q"

# Información del creador
CREATOR_INFO = {
    "name": "Sebastián Sánchez Navarro",
    "email": "inverydesar@gmail.com",
    "project": "Casas para Todos"
}

# Configuración de logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO
)
logger = logging.getLogger(__name__)

In [35]:
def init_database():
    conn = sqlite3.connect('dreambuilder.db')
    c = conn.cursor()

    # Tabla de usuarios
    c.execute('''CREATE TABLE IF NOT EXISTS users
                 (user_id INTEGER PRIMARY KEY,
                  username TEXT,
                  first_name TEXT,
                  last_name TEXT,
                  level INTEGER DEFAULT 1,
                  coins INTEGER DEFAULT 0,
                  experience INTEGER DEFAULT 0,
                  last_daily TIMESTAMP,
                  houses_built INTEGER DEFAULT 0,
                  total_donations REAL DEFAULT 0,
                  referral_code TEXT UNIQUE,
                  referred_by INTEGER,
                  join_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP)''')

    # Tabla de donaciones
    c.execute('''CREATE TABLE IF NOT EXISTS donations
                 (donation_id INTEGER PRIMARY KEY AUTOINCREMENT,
                  user_id INTEGER,
                  amount REAL,
                  timestamp TIMESTAMP,
                  payment_method TEXT,
                  status TEXT,
                  transaction_id TEXT UNIQUE,
                  FOREIGN KEY (user_id) REFERENCES users(user_id))''')

    # Tabla de logros
    c.execute('''CREATE TABLE IF NOT EXISTS achievements
                 (user_id INTEGER,
                  achievement_name TEXT,
                  achieved_date TIMESTAMP,
                  reward_claimed BOOLEAN DEFAULT FALSE,
                  PRIMARY KEY (user_id, achievement_name),
                  FOREIGN KEY (user_id) REFERENCES users(user_id))''')

    # Tabla de construcciones
    c.execute('''CREATE TABLE IF NOT EXISTS constructions
                 (construction_id INTEGER PRIMARY KEY AUTOINCREMENT,
                  user_id INTEGER,
                  building_type TEXT,
                  start_date TIMESTAMP,
                  completion_date TIMESTAMP,
                  status TEXT,
                  progress INTEGER DEFAULT 0,
                  FOREIGN KEY (user_id) REFERENCES users(user_id))''')

    # Tabla de eventos especiales
    c.execute('''CREATE TABLE IF NOT EXISTS special_events
                 (event_id INTEGER PRIMARY KEY AUTOINCREMENT,
                  event_name TEXT,
                  start_date TIMESTAMP,
                  end_date TIMESTAMP,
                  description TEXT,
                  reward_type TEXT,
                  reward_amount INTEGER)''')

    # Tabla de mensajes diarios
    c.execute('''CREATE TABLE IF NOT EXISTS daily_messages
                 (message_id INTEGER PRIMARY KEY AUTOINCREMENT,
                  user_id INTEGER,
                  message_date DATE,
                  message_type TEXT,
                  content TEXT,
                  FOREIGN KEY (user_id) REFERENCES users(user_id))''')

    conn.commit()
    conn.close()

# Inicializar base de datos
init_database()

In [36]:
# Constantes del juego
NIVELES_CONSTRUCCION = {
    1: {
        "nombre": "Casa Básica",
        "costo": 100,
        "recompensa": 50,
        "tiempo_construccion": 300,  # 5 minutos
        "materiales_necesarios": {
            "madera": 10,
            "cemento": 5,
            "ladrillos": 20
        }
    },
    2: {
        "nombre": "Casa Mejorada",
        "costo": 250,
        "recompensa": 125,
        "tiempo_construccion": 600,  # 10 minutos
        "materiales_necesarios": {
            "madera": 20,
            "cemento": 10,
            "ladrillos": 40
        }
    },
    3: {
        "nombre": "Casa de Lujo",
        "costo": 500,
        "recompensa": 300,
        "tiempo_construccion": 1200,  # 20 minutos
        "materiales_necesarios": {
            "madera": 50,
            "cemento": 25,
            "ladrillos": 100
        }
    }
}

# Logros del juego
LOGROS = {
    "primer_donativo": {
        "nombre": "Primer Donativo",
        "descripcion": "Realiza tu primer donativo.",
        "recompensa": 100
    },
    "constructor_experto": {
        "nombre": "Constructor Experto",
        "descripcion": "Construye 10 casas.",
        "recompensa": 500
    }
}

In [37]:
class DreamBuilderGame:
    def __init__(self):
        self.usuarios = {}

    def registrar_usuario(self, user_id, username, first_name, last_name):
        conn = sqlite3.connect('dreambuilder.db')
        c = conn.cursor()
        c.execute('''INSERT OR IGNORE INTO users (user_id, username, first_name, last_name)
                     VALUES (?, ?, ?, ?)''', (user_id, username, first_name, last_name))
        conn.commit()
        conn.close()

    def obtener_perfil(self, user_id):
        conn = sqlite3.connect('dreambuilder.db')
        c = conn.cursor()
        c.execute('SELECT * FROM users WHERE user_id = ?', (user_id,))
        perfil = c.fetchone()
        conn.close()
        return perfil

    def actualizar_monedas(self, user_id, cantidad):
        conn = sqlite3.connect('dreambuilder.db')
        c = conn.cursor()
        c.execute('UPDATE users SET coins = coins + ? WHERE user_id = ?', (cantidad, user_id))
        conn.commit()
        conn.close()

    def construir(self, user_id, nivel):
        if nivel not in NIVELES_CONSTRUCCION:
            return "Nivel de construcción no válido."

        nivel_info = NIVELES_CONSTRUCCION[nivel]
        conn = sqlite3.connect('dreambuilder.db')
        c = conn.cursor()
        c.execute('SELECT coins FROM users WHERE user_id = ?', (user_id,))
        monedas = c.fetchone()[0]

        if monedas < nivel_info["costo"]:
            return "No tienes suficientes monedas para construir esta casa."

        # Restar monedas y registrar construcción
        c.execute('UPDATE users SET coins = coins - ? WHERE user_id = ?', (nivel_info["costo"], user_id))
        c.execute('''INSERT INTO constructions (user_id, building_type, start_date, completion_date, status)
                     VALUES (?, ?, ?, ?, ?)''',
                  (user_id, nivel_info["nombre"], datetime.now(),
                   datetime.now() + timedelta(seconds=nivel_info["tiempo_construccion"]), "En progreso"))
        conn.commit()
        conn.close()
        return f"¡Has comenzado a construir una {nivel_info['nombre']}!"

In [38]:
# Comando /start
def start(update: Update, context: CallbackContext):
    user = update.effective_user
    game.registrar_usuario(user.id, user.username, user.first_name, user.last_name)
    update.message.reply_text(
        f"¡Hola, {user.first_name}! 👋\n"
        "Bienvenido a *Casas para Todos*. Aquí puedes construir casas virtuales, "
        "participar en eventos y ayudar a nuestra causa. 🏠\n\n"
        "Usa /help para ver los comandos disponibles.",
        parse_mode=ParseMode.MARKDOWN
    )

# Comando /help
def help_command(update: Update, context: CallbackContext):
    update.message.reply_text(
        "📋 *Comandos disponibles:*\n\n"
        "/start - Inicia el bot\n"
        "/perfil - Muestra tu perfil\n"
        "/donar - Realiza una donación\n"
        "/eventos - Muestra eventos especiales\n"
        "/logros - Muestra tus logros\n"
        "/referir - Obtén tu código de referido\n"
        "/help - Muestra esta lista de comandos",
        parse_mode=ParseMode.MARKDOWN
    )

# Comando /perfil
def perfil(update: Update, context: CallbackContext):
    user = update.effective_user
    perfil = game.obtener_perfil(user.id)

    if not perfil:
        update.message.reply_text("No se encontró tu perfil. Usa /start para registrarte.")
        return

    mensaje = (
        f"👤 *Tu Perfil*\n\n"
        f"👨‍💻 Usuario: @{perfil[1]}\n"
        f"🏅 Nivel: {perfil[4]}\n"
        f"💰 Monedas: {perfil[5]}\n"
        f"🏠 Casas construidas: {perfil[8]}\n"
        f"🎉 Total donado: ${perfil[9]:.2f}\n"
        f"📅 Fecha de registro: {perfil[11]}"
    )
    update.message.reply_text(mensaje, parse_mode=ParseMode.MARKDOWN)

In [39]:
# Comando /donar
def donar(update: Update, context: CallbackContext):
    user = update.effective_user
    if len(context.args) != 1:
        update.message.reply_text("Por favor, usa el comando así: /donar <cantidad>")
        return

    try:
        cantidad = float(context.args[0])
        if cantidad <= 0:
            update.message.reply_text("La cantidad debe ser mayor a 0.")
            return

        # Registrar donación en la base de datos
        conn = sqlite3.connect('dreambuilder.db')
        c = conn.cursor()
        c.execute('INSERT INTO donations (user_id, amount, timestamp, status) VALUES (?, ?, ?, ?)',
                  (user.id, cantidad, datetime.now(), "Pendiente"))
        conn.commit()
        conn.close()

        # Actualizar monedas del usuario
        game.actualizar_monedas(user.id, int(cantidad * 10))  # 1 donación = 10 monedas
        update.message.reply_text(
            f"¡Gracias por tu donación de ${cantidad:.2f}! 🎉\n"
            f"Has recibido {int(cantidad * 10)} monedas como recompensa. 💰",
            parse_mode=ParseMode.MARKDOWN
        )
    except ValueError:
        update.message.reply_text("Por favor, ingresa una cantidad válida.")

In [40]:
# Comando /eventos
def eventos(update: Update, context: CallbackContext):
    conn = sqlite3.connect('dreambuilder.db')
    c = conn.cursor()
    c.execute('SELECT event_name, description, start_date, end_date FROM special_events WHERE start_date <= ? AND end_date >= ?',
              (datetime.now(), datetime.now()))
    eventos_activos = c.fetchall()
    conn.close()

    if not eventos_activos:
        update.message.reply_text("No hay eventos especiales activos en este momento.")
        return

    mensaje = "🎉 *Eventos Especiales Activos* 🎉\n\n"
    for evento in eventos_activos:
        mensaje += f"📌 *{evento[0]}*\n{evento[1]}\n🗓 Desde: {evento[2]}\n🗓 Hasta: {evento[3]}\n\n"

    update.message.reply_text(mensaje, parse_mode=ParseMode.MARKDOWN)

In [41]:
# Comando /logros
def logros(update: Update, context: CallbackContext):
    user = update.effective_user
    conn = sqlite3.connect('dreambuilder.db')
    c = conn.cursor()
    c.execute('SELECT achievement_name, achieved_date, reward_claimed FROM achievements WHERE user_id = ?', (user.id,))
    logros_usuario = c.fetchall()
    conn.close()

    if not logros_usuario:
        update.message.reply_text("Aún no has desbloqueado ningún logro. ¡Sigue participando!")
        return

    mensaje = "🏆 *Tus Logros* 🏆\n\n"
    for logro in logros_usuario:
        estado = "✅ Reclamado" if logro[2] else "🔓 No reclamado"
        mensaje += f"🎖 *{logro[0]}*\n📅 Fecha: {logro[1]}\nEstado: {estado}\n\n"

    update.message.reply_text(mensaje, parse_mode=ParseMode.MARKDOWN)

In [25]:
# Comando /referir
def referir(update: Update, context: CallbackContext):
    user = update.effective_user
    conn = sqlite3.connect('dreambuilder.db')
    c = conn.cursor()
    c.execute('SELECT referral_code FROM users WHERE user_id = ?', (user.id,))
    referral_code = c.fetchone()[0]

    if not referral_code:
        referral_code = f"REF{user.id}{random.randint(1000, 9999)}"
        c.execute('UPDATE users SET referral_code = ? WHERE user_id = ?', (referral_code, user.id))
        conn.commit()

    conn.close()
    update.message.reply_text(
        f"🔗 *Tu código de referido*: {referral_code}\n"
        "Comparte este código con tus amigos. ¡Ambos recibirán recompensas cuando se registren!",
        parse_mode=ParseMode.MARKDOWN
    )

In [27]:
# Mensajes diarios
def enviar_mensajes_diarios():
    bot = Updater(TOKEN).bot  # Obtener instancia del bot
    conn = sqlite3.connect('dreambuilder.db')
    c = conn.cursor()
    c.execute('SELECT user_id FROM users')
    usuarios = c.fetchall()
    conn.close()

    for usuario in usuarios:
        user_id = usuario[0]
        mensaje = "🌟 ¡Buenos días! No olvides construir y participar en los eventos de *Casas para Todos*."
        try:
            bot.send_message(chat_id=user_id, text=mensaje, parse_mode=ParseMode.MARKDOWN)
        except Exception as e:
            logger.error(f"Error enviando mensaje diario a {user_id}: {str(e)}")

# Programar mensajes diarios
schedule.every().day.at("09:00").do(enviar_mensajes_diarios)

Every 1 day at 09:00:00 do enviar_mensajes_diarios() (last run: [never], next run: 2024-12-25 09:00:00)

In [42]:
# Manejo de errores
def error_handler(update: Update, context: CallbackContext):
    logger.error(msg="Ocurrió un error mientras se procesaba una actualización.", exc_info=context.error)
    update.message.reply_text("⚠️ Ocurrió un error. Por favor, intenta nuevamente.")

In [44]:
def run_schedule():
    while True:
        schedule.run_pending()
        time.sleep(1)

def main():
    print("🤖 Iniciando el bot...")
    updater = Updater(TOKEN)
    dispatcher = updater.dispatcher

    # Registrar comandos
    dispatcher.add_handler(CommandHandler("start", start))
    dispatcher.add_handler(CommandHandler("help", help_command))
    dispatcher.add_handler(CommandHandler("perfil", perfil))
    dispatcher.add_handler(CommandHandler("donar", donar))
    dispatcher.add_handler(CommandHandler("eventos", eventos))
    dispatcher.add_handler(CommandHandler("logros", logros))
    dispatcher.add_handler(CommandHandler("referir", referir))

    # Manejo de errores
    dispatcher.add_error_handler(error_handler)

    print("✅ Comandos registrados correctamente")
    return updater

# Crear instancia del juego
game = DreamBuilderGame()
print("✅ Juego inicializado")

✅ Juego inicializado


In [57]:
# Ejecutar el bot
print("🚀 Iniciando el bot de Casas para Todos...")
updater = main()
print("✅ Bot configurado correctamente")
print("⚡ Iniciando polling...")
updater.start_polling()
print("🎮 Bot activo y funcionando!")
print("💡 Puedes usar el bot en Telegram ahora.")
print("⚠️ Para detener el bot, presiona el botón 'Stop' en la celda o reinicia el runtime")

🚀 Iniciando el bot de Casas para Todos...
🤖 Iniciando el bot...
✅ Comandos registrados correctamente
✅ Bot configurado correctamente
⚡ Iniciando polling...
🎮 Bot activo y funcionando!
💡 Puedes usar el bot en Telegram ahora.
⚠️ Para detener el bot, presiona el botón 'Stop' en la celda o reinicia el runtime
